## Using the N-Body Method

Having written an n-body right hand side function and bound the gravitational acceleration, lets try it out!

Begin by resurrecting your ode code and doing some runs with initial conditions provided below. We know our best bet will be the Euler-Richardson algorithm. Integrate through 100 time units and plot the result.

I've left plotting code to save you some time with that.

For each of the initial conditions, find the maximum allowable time step the allows visually acceptable repeating orbits for 100 time steps. Record those time steps for future reference.

After completing that exercise, implement the fourth order Runge Kutta algorithm as part of your ODE solving function and repeat the above exercise. Are you doing better in terms of time steps? What about the number of function calls.

**Note:** I've tried to save you the hassle of writing plotting code. I figure you've done enough of that this semester. The problem is, I don't think mine is especially tidy. If you feel like replacing mine with something better, by all means, go for it.

### Runge Kutta

The simplest but steadiest performance can be realized with the *fourth order Runge-Kutta* integration methods. The following equations haver errors of order $\mathcal{O}(\Delta t^5)$. They can be expressed as.

$$
\begin{align}
k_1 &=& &\Delta t f(t_n,y_n)\\
k_2 &=& &\Delta t f(t_n + \frac{1}{2}\Delta t,y_n + \frac{1}{2} k_1)\\
k_3 &=& &\Delta t f(t_n + \frac{1}{2}\Delta t,y_n + \frac{1}{2} k_2)\\
k_4 &=& &\Delta t f(t_n + \frac{1}{2}\Delta t,y_n +  k_3)
\end{align}
$$

$$y_{n+1} = y_n + \frac{1}{6} k_1 + \frac{1}{3} k_2 + \frac{1}{3} k_3 + \frac{1}{6} k_4 $$

In [1]:
def RK45( y, dt, f, t, *p, **options):

    S=.9998
    b = np.array([35 / 384, 0, 500 / 1113, 125 / 192, -2187 / 6784, 11 / 84, 0])
    b_star = np.array(                [5179 / 57600, 0, 7571 / 16695, 393 / 640, -92097 / 339200, 187 / 2100, 1 / 40])
    c = np.array([0,0.2, 0.3, 0.8, 8.0 / 9.0,1,1])
    a = np.array([[0, 0, 0, 0, 0, 0],
                 [1 / 5, 0, 0, 0, 0, 0],
                 [3 / 40, 9 / 40, 0, 0, 0, 0],
                 [44 / 45, -56 / 15, 32 / 9, 0, 0, 0],
                 [19372 / 6561, -25360 / 2187, 64448 / 6561, -212 / 729, 0, 0],
                 [9017 / 3168, -355 / 33, 46732 / 5247, 49 / 176, -5103 / 18656, 0],
                 [35 / 384, 0, 500 / 1113, 125 / 192, -2187 / 6784, 11 / 84]])
    FSAL = options['FSAL']
    last_k = options['last_k']
    atol = options['atol']
    rtol = options['rtol']
    if FSAL:
        k1 = last_k
    else:
        k1 = dt * f(t, y, *p)
    y_2 =  y + a[1, 0] * k1
    k2 = dt * f(t + c[0] * dt,y_2, *p)
    y_3 =  y + a[2, 0] * k1 + a[2, 1] * k2
    k3 = dt * f(t + c[1] * dt,y_3, *p)
    y_4 = y + a[3, 0] * k1 + a[3, 1] * k2 + a[3, 2] * k3
    k4 = dt * f(t + c[2] * dt, y_4, *p)
    y_5 = y + a[4, 0] * k1 + a[4, 1] * k2 + a[4, 2] * k3 + a[4, 3] * k4
    k5 = dt * f(t + c[3] * dt, y_5, *p)
    y_6 = y + a[5, 0] * k1 + a[5, 1] * k2 + a[5, 2] * k3 + a[5, 3] * k4 + a[5, 4] * k5
    k6 = dt * f(t + dt,y_6, *p)
    y_7 =  y + a[6, 0] * k1 + a[6, 1] * k2 + a[6, 2] * k3 + a[6, 3] * k4 + a[6, 4] * k5+ a[6, 5] * k6
    k7 = dt * f(t + dt,y_7, *p)


    k = np.row_stack([k1, k2, k3, k4, k5, k6, k7])
    y_new = y + np.dot(b, k)

    delta = np.dot(b - b_star, k)
    scale = atol + rtol * np.abs(y_new)
    error = np.sqrt(np.mean((delta / scale) ** 2))

    dt_new = S * dt * np.power(1 / error, 1 / 5)

    if error > 1:
        options['FSAL'] = False
        options['last_k'] = 0
        dt_new = max(dt_new, dt / 5)
        y_new, dt_new, FSAL,last_k = RK45(y, dt_new, f, t, *p, **options)
    else:
        dt_new = min(dt_new, dt * 10)
        FSAL = True
        last_k = k7

    return y_new, dt_new,FSAL,last_k

SyntaxError: invalid syntax (<ipython-input-1-62d7fc9e06bc>, line 2)

In [ ]:
import matplotlib.pyplot as plt

def plot_KE_and_PE(y,p,t_s):
    KE = []
    PE = []
    m = len(p['m'])
    position = y[:,:y[0].size//2]
    velocity = y[:,y[0].size//2:]
    for i in range(len(y)):
        KE.append(np.sum(np.power(velocity[i],2)/2.))
        temp = []
        for  j in range(m):
                for k in range(j+1,m):
                    dx = position[i,j*d]-position[i,k*d]
                    dy = position[i,1+j*d]-position[i,1+k*d]
                    r_inverse =np.sqrt(dx**2+dy**2)
                    temp.append(1./r_inverse)
        PE.append(-np.sum(temp))

    plt.plot(t_s,KE)
    plt.plot(t_s,PE)
    plt.show()
    plt.plot(t_s,np.add(KE,PE))
    plt.show()

In [ ]:

def Euler(f,dt,t,y,args):
    """
    given y(t)
    :return: y(t+delta_t)
    """
    return y + f(t,y,*args)* dt,dt

def EulerCromer(f,dt,t,y,args):
    y_end = Euler(f,dt,t,y,args)
    return y + f(t+dt,y_end,*args) * dt,dt


def EulerRichardson(f,dt,t,y,args):
    y_mid = Euler(f,dt,t+dt/2,y,args)
    return y + f(t+dt/2,y_mid,*args) * dt,dt


def solve_ode(f,tspan, y0, method = Euler, *args, **options):
    t_0 = tspan[0]
    t_f = tspan[1]
    d_t = options['first_step']
    y = []
    t = []
    t.append(tspan[0])
    y.append(y0)
    dt = options['first_step']
    option_list = [options]
    dt_list = [dt]
    if method in (Euler,EulerCromer,EulerRichardson,runge_kutta4):

        while t[-1] < tspan[1]:
            y_i,dt = method(f,t[-1],y[-1],dt,args,options)
            y.append(y_i)
            t.append(t[-1]+dt)
            option_list.append(options)
            dt_list.append(dt)
    elif method is RK45:
        options['FSAL'] = False
        options['last_k'] =0
        while t[-1] < tspan[1]:
            current_t = t[-1]
            new_y, dt,options['FSAL'],options['last_k'] = method(y[-1], dt, f, t[-1], *args, **options)
            y.append(new_y)
            t.append(t[-1] + dt)
            dt_list.append(dt)

    return np.array(t),np.array(y),option_list,dt_list

In [4]:

import numpy as np
def gravity(rij, m,G):
    radious = np.linalg.norm(rij)
    return (-G * m[0] *m[1] *rij)/(radious**3)

def n_body(t,y,p):
    '''
    p={masses: { floats }}
    x = [x1,y1,x2,y2  ....,vx1,vy1,]
    Write what goes in here!
    Instructions above.
    '''

    dim = p['dimension']
    masses = p['m']
    N = len(masses)
    G_const =  p['G']
    # force = p['force'] # force function
    half =y.size //2
    accel = np.zeros((N, N, dim))

    # print(y.shape)
    for i in range(0, y.size//2, dim):
        for j in range(i + dim, y.size//2, dim):
            rij = y[i:i+dim] - y[j:j+dim]
            f = gravity(rij, [masses[i//dim], masses[j//dim]],G_const)
            accel[i//dim][j//dim] = f / masses[i//dim]
            accel[j//dim][i//dim] = - f / masses[j//dim]
            
            
    dv_dt = np.sum(accel, axis=1).flatten().tolist()
    dy_dt = y[y.size//2:]
#     print('dv,dy',dv_dt)
    dxdt =  np.array([dy_dt,dv_dt]).flatten()
    # print(f'len {len(dxdt)}',dxdt)
    if p['fix_first']:
        dxdt[:dim] = 0
        dxdt[half: half+dim] = 0
    return dxdt


def runge_kutta4(dt, f, y, t, p):
    k1 = dt*f(t,y,p)
    k2 = dt*f(t+.5*dt,y+.5*k1,p)
    k3 = dt*f(t+.5*dt,y+.5*k2,p)
    k4 =  dt*f(t+dt,y+k3,p)
    rk4 = (k1 + 2 * k2 + 2 * k3 + k4)/6
    return rk4

import numpy as np
# Order is all coordinates then all velocities in groups by mass:
# x1,y1,x2,y2,x3,y3,vx1,vy1,vx2,vy2,etc
euler      = np.array([0,0,1,0,-1,0,0,0,0,.8,0,-.8])

montgomery = np.array([0.97000436,-0.24308753,-0.97000436,0.24308753, 0., 0.,\
                    0.466203685, 0.43236573, 0.466203685, 0.43236573,\
                   -0.93240737,-0.86473146])
lagrange   = np.array([1.,0.,-0.5,0.866025403784439, -0.5,-0.866025403784439,\
                  0.,0.8,-0.692820323027551,-0.4, 0.692820323027551, -0.4])

p3 = {'m':[1,1,1],'G':1,'dimension':2,'acceleration':gravity,'fix_first':False}
p4 = {'m':[1,1,1,1],'G':1,'dimension':2,'fix_first':False}
phe = {'m':[2,-1,-1],'G':1,'dimension':2,'fix_first':True}

In [5]:
import numpy as np
euler      = np.array([0,0,1,0,-1,0,0,0,0,.8,0,-.8])
four_body  = np.array([1.382857,0,\
                   0,0.157030,\
                  -1.382857,0,\
                   0,-0.157030,\
                   0,0.584873,\
                   1.871935,0,\
                   0,-0.584873,\
                  -1.871935,0],dtype=np.float128)
helium_1 = np.array([0,0,2,0,-1,0,0,0,0,.95,0,-1])



import matplotlib.pyplot as plt
from matplotlib import animation,rc
from IPython.display import HTML

plt.style.use('dark_background')
%matplotlib inline

y0 = four_body
p  = p4
dt = .01  # This is wrong - figure it out!
t_span = [0,100]

t_s,y = solve_ode(n_body,t_span, y0, EulerRichardson, p,first_step=dt)

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)

d = p['dimension']
x_min,x_max,y_min,y_max = 1e9,-1e9,1e9,-1e9
for i in range(0,y0.size//d,d):
    x_t = y[:,i]
    y_t = y[:,i+1]
    if x_min > x_t.min(): x_min = x_t.min()
    if x_max < x_t.max(): x_max = x_t.max()
    if y_min > y_t.min(): y_min = y_t.min()
    if y_max < y_t.max(): y_max = y_t.max()

    ph, =  ax.plot(x_t,y_t,'-',color=[.7,.7,.7],linewidth=.5) 

plt.xlim([1.2*x_min,1.2*x_max])
plt.ylim([1.2*y_min,1.2*y_max])

ax.axis('off')
plt.show()

NameError: name 'solve_ode' is not defined

In [ ]:
d = p['dimension']
trace_length = 20
c=['tab:red','tab:olive','tab:pink','tab:cyan','tab:purple']
body_list = []
trace_list = []

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
FRAMES = 500
x_min,x_max,y_min,y_max = 1e9,-1e9,1e9,-1e9
for i in range(0,y0.size//d,d):
    x_t = y[:,i]
    y_t = y[:,i+1]
    if x_min > x_t.min(): x_min = x_t.min()
    if x_max < x_t.max(): x_max = x_t.max()
    if y_min > y_t.min(): y_min = y_t.min()
    if y_max < y_t.max(): y_max = y_t.max()

    ph, =  ax.plot(x_t,y_t,'-',color=[.7,.7,.7],linewidth=.5) 

plt.xlim([1.2*x_min,1.2*x_max])
plt.ylim([1.2*y_min,1.2*y_max])

ax.axis('off')

for i in range(0,y0.size//d,d):
    ph, =  ax.plot(y0[i],y0[i+1],'o',color=c[i//d]); 
    body_list.append( ph )
    ph, = ax.plot([],[],'-',color=c[i//d])
    trace_list.append( ph )

def animate(i):
    i = i % t_s.size
    for im,j in zip(body_list,range(0,d*len(body_list),d)):
        im.set_xdata( y[i+1,j] )
        im.set_ydata( y[i+1,j+1] )

    if i>trace_length:
        for im,j in zip(trace_list,range(0,d*len(trace_list),d)):
            im.set_xdata( y[i-trace_length:i+1,j] )
            im.set_ydata( y[i-trace_length:i+1,j+1] )
    return im

anim = animation.FuncAnimation(fig, animate, interval=20,frames=FRAMES) 

HTML(anim.to_html5_video())